# Volatility Regime Prediction Checkpoint 2

## Overview
For the second part of our project, we focused on improving our volatilty regime prediction algorithm and implementing a portfolio trading strategy using the data from our regime prediction algorithm. The trading strategy considers a diverse portfolio of ETFs. When our regime prediction says we will be in a low-vol environment next week, then our strategy uses equal weighting to our portfolio; however, when the prediction for next week's volatility is high-vol, then the portfolio adjusts from equal weighting to min-vol weighting. To find the min-vol weighting we used the scipy.optimize package, and leveraged the sco.minimize function to find weights that minimize volatilty over the previous two weeks.

With the random forest, our algorithm provides probabilities for a high-vol regime and low-vol regime for next week. Instead of investing all money into our high/low vol portfolios, we invest a percent based on the probability the random forest gives us that next week will either be high/low vol.

## Portfolio Class 
The Portfolio class that we created takes in start_date, end_date, tickers, and an object from our Regime_Predict class. Within the class, there is a weekly_optimization function that performs our trading strategy by investing proper amounts into our min_vol and equally weighted portfolios. The file also contains various portfolio functions such as functions to create returns matrices, calculate portfolio returns/vol, and the function that uses sco.minimize to find the minimum vol weights for a portfolio. 

## ETFs For Portfolio
#### - PNQI: NASDAQ Internet ETF  
#### - SPY: SPDR S&P 500 ETF Trust
#### - SPXL: Direxion Daily S&P 500 Bull 3X
#### - XLK: Technology Select Sector SPDR Fund
#### - XLY: Consumer Discretionary SPDR
#### - XLF: Financial Select Sector SPDR Fund
#### - SHY: iShares Barclay 1-3 Year Treasury Bond Fund


## Vol_Outputs Class
Within our main.py file, we have a Vol_Outputs class that is meant to tie together all of our other classes to provide outputs. Vol_Outputs takes in start_date, end_date, tickers, and fred_strings. From here, in its constructor, the class creates Vol_Data, Regime_Predict, and Portfolio objects which we use to find outputs. 

The Vol_Outputs class currently contains compare_equal, compare_spy, and analyze_volatile_periods methods. Each of these methods is defined to show users how our strategy performs against various benchmarks.
    1. compare_equal: compares how our strategy did against a strategy that equally weights our tickers the entire time
    2. compare_spy: compares how our strategy did against the SPY
    3. analyze_volatile_periods: Shows how our strategy performed during the market's most volatile periods compared to the equally weighted portfolio and the SPY 

In [11]:
import os
from Code.main.main import Vol_Outputs

start_date = "2008-01-01"
end_date="2020-06-20"
tickers = ["PNQI", "SPY","SPXL", "XLK", "XLY", "XLF","SHY"]
fred_strings=["DCOILBRENTEU" ,"BAMLH0A0HYM2", "GOLDAMGBD228NLBM","DAAA","RIFSPPFAAD01NB","BAMLHE00EHYIOAS", "DEXCHUS", "DEXUSEU", "T10Y3M", "BAMLEMFSFCRPITRIV"]
v_trial = Vol_Outputs(start_date, end_date, tickers, fred_strings)

2919
2922


#### The code above creates a Vol_Outputs object called v_trial. From v_trial, we can generate the outputs that we want.

In [12]:
v_trial.compare_equal()
v_trial.compare_spy()
v_trial.analyze_volatile_periods()

{0:                                  Optimized_Port  Equal_Weight_Port       SPY
 Metrics 2009-01-26 - 2009-03-26                                             
 Return                                 0.513810           0.255883  0.000000
 Volatilty                              0.112154           0.506826  0.417252
 VaR                                   -0.184476          -0.833654 -0.686319,
 1:                                  Optimized_Port  Equal_Weight_Port       SPY
 Metrics 2010-05-26 - 2010-07-26                                             
 Return                                 0.005301           0.208752  0.204596
 Volatilty                              0.042142           0.293115  0.245622
 VaR                                   -0.069318          -0.482130 -0.404013,
 2:                                  Optimized_Port  Equal_Weight_Port       SPY
 Metrics 2011-07-26 - 2011-09-26                                             
 Return                                -0.166768     